In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        (os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install -q torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric \
-f https://data.pyg.org/whl/torch-2.1.0+cu121.html


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.0 MB/s eta 0:00:00


In [1]:
import torch
import torch_geometric

print("Torch version:", torch.__version__)
print("Torch Geometric version:", torch_geometric.__version__)


ModuleNotFoundError: No module named 'torch_geometric'

In [ ]:
# Install PyTorch Geometric dependencies
!pip install -q torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.1 MB/s eta 0:00:0000:01


In [ ]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import timm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

from torch.utils.data import Dataset, DataLoader
from torch_geometric.nn import GATConv


In [ ]:
DATASET_DIR = "/kaggle/input/brain-tumor-mri-dataset/Training"
IMG_SIZE = 224
NUM_CLASSES = 4
BATCH_SIZE = 16
EPOCHS = 20
LR = 1e-4

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
def load_dataset_numpy(dataset_dir, img_size):
    images = []
    labels = []

    for label in os.listdir(dataset_dir):
        class_path = os.path.join(dataset_dir, label)
        if not os.path.isdir(class_path):
            continue

        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (img_size, img_size))
            images.append(img)
            labels.append(label)

    images = np.array(images, dtype=np.float32) / 255.0
    labels = np.array(labels)

    return images, labels


In [ ]:
X, y = load_dataset_numpy(DATASET_DIR, IMG_SIZE)
print("Images shape:", X.shape)
print("Labels shape:", y.shape)


In [ ]:
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42
)


In [ ]:
class NumpyDataset(Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = torch.tensor(self.images[idx]).permute(2, 0, 1)
        label = torch.tensor(self.labels[idx])
        return img, label


In [ ]:
train_dataset = NumpyDataset(X_train, y_train)
test_dataset  = NumpyDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [ ]:
class Swin_LSTM_GAT(nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        # Swin Transformer
        self.swin = timm.create_model(
            "swin_tiny_patch4_window7_224",
            pretrained=True,
            num_classes=0
        )

        # LSTM
        self.lstm = nn.LSTM(
            input_size=768,
            hidden_size=512,
            batch_first=True
        )

        # Graph Attention Network
        self.gat1 = GATConv(512, 256, heads=4)
        self.gat2 = GATConv(256 * 4, 128, heads=1)

        # Classifier
        self.fc = nn.Sequential(
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        B = x.size(0)

        swin_feat = self.swin(x)           # (B, 768)
        swin_feat = swin_feat.unsqueeze(1)

        lstm_out, _ = self.lstm(swin_feat) # (B, 1, 512)
        lstm_feat = lstm_out.squeeze(1)

        # Fully connected graph
        edge_index = torch.combinations(
            torch.arange(B), r=2
        ).t().to(x.device)

        gat_out = self.gat1(lstm_feat, edge_index)
        gat_out = torch.relu(gat_out)
        gat_out = self.gat2(gat_out, edge_index)

        return self.fc(gat_out)


In [ ]:
model = Swin_LSTM_GAT(NUM_CLASSES).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)


In [ ]:
def train():
    model.train()
    for epoch in range(EPOCHS):
        total_loss = 0
        for imgs, labels in train_loader:
            imgs, labels = imgs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch [{epoch+1}/{EPOCHS}], Loss: {total_loss:.4f}")


In [ ]:
model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for imgs, labels in test_loader:
        imgs = imgs.to(device)
        outputs = model(imgs)
        preds = torch.argmax(outputs, dim=1).cpu().numpy()
        y_pred.extend(preds)
        y_true.extend(labels.numpy())

print("Accuracy:", accuracy_score(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=encoder.classes_))
